In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pydicom as di
import pandas as pd
import tensorflow as tf
from PIL import Image
import pydicom as pydcm
import cv2

In [ ]:
# Loading the CSV file and cleaning the data
train_csv = pd.read_csv("./train.csv")
train_csv.head(10)

In [ ]:
#Exploring data
print(f"<-- Difficult negative case --> \n {train_csv['difficult_negative_case'].value_counts()}\n")

print(f"<-- Cancer --> \n {train_csv['cancer'].value_counts()}\n")

# The data is very skewed.

In [ ]:
# Getting a csv of the files we have.
patient_ids_on_hand = []
#Getting the names of the folders at hand.
for file in os.listdir("./images"):
    patient_ids_on_hand.append(int(file))
len(patient_ids_on_hand)

In [ ]:
# Creating a CSV of images on hand.
csv_data_on_hand = pd.DataFrame(columns=train_csv.columns)

for patient_id in patient_ids_on_hand:
   csv_data_on_hand =  pd.concat([csv_data_on_hand, train_csv[train_csv['patient_id'] == patient_id]])


In [ ]:
# Looking at distribution of postivie values.
csv_data_on_hand["cancer"].value_counts()

In [ ]:
#Fucntion to convert dcm images to nparray of type uint16
def dcm_to_numpy(patient_id, image_id):
    path_to_images = "./images/"
    path_to_images += str(patient_id) + "/" + str(image_id) + ".dcm"
    dcm_image = di.dcmread(path_to_images)
    np_array = dcm_image.pixel_array
    return np_array

#function to save dcm image as png
def save_dcm_as_png(path_to_dcm, png_save_path):
    im = Image.fromarray(dcm_to_numpy(path_to_dcm)*128)
    im.save(png_save_path)

# takes df with info of a single patient and concats the corresponding images into one
def image_concat(patient_df):
    print(patient_id)
    image_array = []
    for image_id in patient_df[patient_df['laterality'] == 'L']['image_id']:
        image_array.append(dcm_to_numpy(patient_id, image_id))
    
    row1 = np.concatenate((image_array[0], image_array[1]), axis = 1) # concat L images

    for image_id in patient_df[patient_df['laterality'] == 'R']['image_id']:
        image_array.append(dcm_to_numpy(patient_id, image_id))

    row2 = np.concatenate((image_array[0], image_array[1]), axis = 1) # concat R images

    final_image = np.concatenate((row1, row2), axis=0) # concat L and R images

    # im = Image.fromarray(final_image)
    # im = im.convert('RGB')
    # im.save('./concat_images/{}.png'.format(patient_id), compress_level = 1)

    cv2.imwrite('./concat_images/{}.png'.format(patient_id), final_image)

    #####
    # processing is very slow for some reason
    #####

In [ ]:
# iterate over each unique patient id
for patient_id in csv_data_on_hand['patient_id'].unique():
    current_patient = csv_data_on_hand[csv_data_on_hand['patient_id'] == patient_id]
    image_concat(current_patient)